In [1]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display, Image
import shutil
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
with open('test_ids.txt', 'r') as file:
    # Read all lines into a list
    lines = file.readlines()
    test_ids = []

    # Print each line
    for line in lines:
        test_ids.append(line.strip() + '.jpg')

In [3]:
with open('train_ids.txt', 'r') as file:
    # Read all lines into a list
    lines = file.readlines()
    train_ids = []

    # Print each line
    for line in lines:
        train_ids.append(line.strip() + '.jpg')

In [4]:
with open('val_ids.txt', 'r') as file:
    # Read all lines into a list
    lines = file.readlines()
    val_ids = []

    # Print each line
    for line in lines:
        val_ids.append(line.strip() + '.jpg')

In [5]:
print("Number of images in training dataset: ", len(train_ids))
print("Number of images in validation dataset: ", len(val_ids))
print("Number of images in testing dataset: ", len(test_ids))

Number of images in training dataset:  134823
Number of images in validation dataset:  5000
Number of images in testing dataset:  10000


In [6]:
#get image files
def get_image_files(folder_path):
    files = os.listdir(folder_path)
    image_files = []
    
    for file in files:
        if file.lower().endswith('.jpg'):
            image_files.append(file)
    
    return image_files

In [7]:
train_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/train'
test_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/test'
val_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/valid'

In [8]:
def copy_files(source_path, destination_path, titles):
    for title in titles:
        source_file = os.path.join(data_directory, title)
        destination_file = os.path.join(destination_path, title)
        shutil.copy(source_file, destination_file)

In [9]:
#data_directory = "/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/img_resized/"

In [10]:
#already done
#copy_files(data_directory, train_path, train_ids)
#copy_files(data_directory, val_path, val_ids)
#copy_files(data_directory, test_path, test_ids)

In [11]:
len(get_image_files(train_path))

0

In [12]:
import json

# Specify the path to your JSON file
json_file_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/MMHS150K_GT.json'

# Open the JSON file for reading
with open(json_file_path, 'r') as json_file:
    # Load the JSON data
    data = json.load(json_file)

In [13]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df.reset_index()
newColumns = {"index": "id"}
df = df.rename(columns=newColumns)

In [14]:
#0 for nothate, #1 for hate
def labels_to_binary(labels):
    if sum(labels) <= 1:
        return 0
    else:
        return 1

In [15]:
df["Hate(1)_vs_NotHate(0)"] = df["labels"].apply(labels_to_binary)
sum(df["Hate(1)_vs_NotHate(0)"]==1)

61547

In [16]:
nothate_ids = df[df["Hate(1)_vs_NotHate(0)"]==0]["id"].tolist()

In [17]:
hate_ids = df[df["Hate(1)_vs_NotHate(0)"]==1]["id"].tolist()

In [18]:
nothate_ids = [x + '.jpg' for x in nothate_ids]
hate_ids = [x + '.jpg' for x in hate_ids]

In [19]:
"""hate_train_ids = []
hate_val_ids = []
hate_test_ids = []
for id_ in hate_ids:
    if id_ in train_ids:
        hate_train_ids.append(id_)
    elif id_ in test_ids:
        hate_test_ids.append(id_)
    else:
        hate_val_ids.append(id_)""";

In [20]:
"""nothate_train_ids = []
nothate_val_ids = []
nothate_test_ids = []
for id_ in nothate_ids:
    if id_ in train_ids:
        nothate_train_ids.append(id_)
    elif id_ in test_ids:
        nothate_test_ids.append(id_)
    else:
        nothate_val_ids.append(id_)""";

In [21]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.applications import ResNet50
#from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import cv2
#from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
dataset_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project'

# Define the image size based on the pre-trained model's requirements
img_size = (224, 224)

# Define the batch size
batch_size = 32


def preprocess_with_denoising(image):
    # Gaussian Blur for noise reduction
    denoised_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Original preprocess_input function
    preprocessed_image = preprocess_input(denoised_image)

    return preprocessed_image

# ImageDataGenerator with noise reduction in preprocessing
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_with_denoising,
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [22]:
# Create data generators for training, validation, and test sets
train_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',  # Set to 'binary' for binary classification
    shuffle=False  # set shuffle to False for feature extraction
)


validation_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, 'valid'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 58954 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [30]:
from tensorflow.keras.applications import VGG16

# Local path to save the weights file
local_weights_path = '/Users/raghavraahul/Downloads/'

# Download and save the weights of the entire VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_model.save_weights(local_weights_path + 'vgg16_weights.h5')

In [31]:
from tensorflow.keras.applications import MobileNetV2


# Create the model with the correct architecture
base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))

# Modify the number of filters in the first convolutional layer

#This modification assumes that the original weights file had 64 filters, 
#and you're adjusting it to match the current model with 32 filters. 
#base_model.layers[1].set_weights([base_model.layers[1].get_weights()[0][:, :, :, :32]])

# Load the modified weights
base_model.load_weights(local_weights_path + 'vgg16_weights.h5')

In [32]:
# Extract features from images using the pre-trained model
train_features = base_model.predict(train_generator, steps=len(train_generator))
validation_features = base_model.predict(validation_generator, steps=len(validation_generator))
test_features = base_model.predict(test_generator, steps=len(test_generator))

313/313 [==============================] - 8361s 27s/step


In [33]:
# Flatten the features
train_features = np.reshape(train_features, (len(train_generator.classes), -1))
validation_features = np.reshape(validation_features, (len(validation_generator.classes), -1))
test_features = np.reshape(test_features, (len(test_generator.classes), -1))

In [34]:
np.savez('VGG16_features.npz', train=train_features, validation=validation_features, test=test_features)

In [35]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Load the extracted features

loaded_arrays = np.load('VGG16_features.npz')

# Access individual arrays using keys
train_features = loaded_arrays['train']
validation_features = loaded_arrays['validation']
test_features = loaded_arrays['test']

#train_features = np.load('imagenet_train_features.npy')
#validation_features = np.load('imagenet_validation_features.npy')
#test_features = np.load('imagenet_test_features.npy')

# Load the corresponding labels
train_labels = train_generator.classes
validation_labels = validation_generator.classes
test_labels = test_generator.classes

In [36]:
# Define and compile the custom model
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=train_features.shape[1])) #input

model.add(Dense(128, activation='relu', input_dim=train_features.shape[1]))
model.add(Dense(64, activation='relu', input_dim=train_features.shape[1]))
model.add(Dense(32, activation='relu', input_dim=train_features.shape[1]))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid')) #output

In [37]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy', AUC()])

In [38]:
# Train the model on the extracted features
history = model.fit(train_features, train_labels, 
                    epochs=10, 
                    batch_size=32,
                    validation_data=(validation_features, validation_labels))

Epoch 1/10
1843/1843 [==============================] - 67s 35ms/step - loss: 0.6970 - accuracy: 0.4975 - auc: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5000 - val_auc: 0.5000
Epoch 2/10
1843/1843 [==============================] - 62s 34ms/step - loss: 0.6932 - accuracy: 0.5001 - auc: 0.5013 - val_loss: 0.6931 - val_accuracy: 0.5000 - val_auc: 0.5000
Epoch 3/10
1843/1843 [==============================] - 64s 34ms/step - loss: 0.6932 - accuracy: 0.5005 - auc: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.5000 - val_auc: 0.5000
Epoch 4/10
1843/1843 [==============================] - 64s 35ms/step - loss: 0.6947 - accuracy: 0.4981 - auc: 0.4969 - val_loss: 0.6931 - val_accuracy: 0.5000 - val_auc: 0.5000
Epoch 5/10
1843/1843 [==============================] - 67s 36ms/step - loss: 0.6932 - accuracy: 0.4986 - auc: 0.4980 - val_loss: 0.6931 - val_accuracy: 0.5000 - val_auc: 0.5000
Epoch 6/10
1843/1843 [==============================] - 64s 35ms/step - loss: 0.6931 - accuracy: 0.5019 - auc:

In [39]:
# Evaluate the model on the test set
test_loss, test_acc, test_auc = model.evaluate(test_features, test_labels)
print(f'Test accuracy: {test_acc * 100:.2f}%')

313/313 [==============================] - 4s 11ms/step - loss: 0.6932 - accuracy: 0.5001 - auc: 0.5000
Test accuracy: 50.01%


In [40]:
test_acc

0.5001000165939331

In [41]:
test_auc

0.5

In [42]:
raw_predictions = model.predict(test_features)

# Convert raw predictions to binary labels using a threshold (e.g., 0.5)
threshold = 0.5
predicted_labels = (raw_predictions > threshold).astype(int)
raw_predictions = [array[0] for array in raw_predictions]


predicted_labels

313/313 [==============================] - 4s 12ms/step


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [43]:
processed_predicted_labels = [array[0] for array in predicted_labels]

In [44]:
df = pd.DataFrame(
    {
        "predicted_labels": processed_predicted_labels
    }
)

In [45]:
df.to_excel("CNN_run3.xlsx", index=False)

In [46]:
test_loss

0.6931775212287903

In [ ]:
"""import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create a Keras model
def create_model(learning_rate=0.001, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_dim=train_features.shape[1]))
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define the hyperparameters to search
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.0, 0.2, 0.4]
}

# Create the grid search object
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=StratifiedKFold(n_splits=3), verbose=1)

# Fit the grid search to the data
grid_result = grid.fit(train_features, train_labels)

# Print the best parameters and corresponding accuracy
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: {:.2f}%".format(grid_result.best_score_ * 100))

# Access the best model from the grid search
best_model = grid_result.best_estimator_.model""";